In [1]:
import os
import utils
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

entire dataset (unlabeled)

In [3]:
dir_path = '../../model_output/'
if not os.path.exists(dir_path):
    raise FileNotFoundError(f"The directory {dir_path} does not exist.")

In [4]:
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
dfs = []

# loop through CSV files and append them to the dfs list
for file in csv_files:
    file_path = os.path.join(dir_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# combine to a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

In [5]:
combined_df.head()

,index,annotation,non-parasite output,parasite output,unsure output
0,95460,-1,2.186631e-07,0.999920,0.000080
1,174898,-1,5.829984e-07,0.999800,0.000200
2,153849,-1,6.457012e-06,0.999766,0.000228
3,108502,-1,6.806655e-07,0.999699,0.000300
4,129384,-1,1.702819e-06,0.999681,0.000317


In [10]:
combined_df['annotation'].unique() # none of these have labels

array([-1])

In [ ]:
lab = Datalab(data=combined_df, label_name="annotation")
lab.find_issues(pred_probs=pred_probs)
lab.report()  # summarize issues in dataset, how severe they are, ...

combined_images_parasite dataset

In [6]:
dir_in = '../'
pos_image_path = 'combined_images_parasite'
model_path = 'model_perf_r34_b32'

if torch.cuda.is_available():
    loaded_model = torch.load(dir_in + model_path + '.pt')
else:
    loaded_model = torch.load(dir_in + model_path + '.pt',map_location=torch.device('cpu'))

pos_images = np.load(dir_in + 'data/' + pos_image_path + '.npy')

/home/squid/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
preds, features = utils.generate_predictions_and_features(loaded_model,pos_images,32)

running inference on 76126 images took 94.34033226966858 s


In [9]:
np.save('pos_preds.npy', preds)
np.save('pos_features.npy', features)

new code (for testing purposes only, needs to have out-of-sample preds from CV)

In [7]:
dir_in = '../'
pos_image_path = 'combined_images_parasite'
model_path = 'model_perf_r34_b32'

if torch.cuda.is_available():
    loaded_model = torch.load(dir_in + model_path + '.pt')
else:
    loaded_model = torch.load(dir_in + model_path + '.pt',map_location=torch.device('cpu'))

pos_images = np.load(dir_in + 'data/' + pos_image_path + '.npy')

/home/squid/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
preds = np.load('pos_preds.npy')
features = np.load('pos_preds.npy')
labels = np.ones(len(preds))

In [11]:
data = {'image': pos_images, 'label': np.ones(len(pos_images))}

In [12]:
from cleanlab import Datalab

# Create a Datalab instance
lab = Datalab(data=data, label_name="label", image_key="image")

# Find issues in the dataset
issues = lab.find_issues(features=features, pred_probs=preds)

Finding label issues ...
Error in label: Labels must contain at least 2 classes.
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Failed to check for these issue types: [LabelIssueManager]
Finding dark, light, low_information, odd_aspect_ratio, odd_size, grayscale, blurry images ...


  0%|          | 0/76126 [00:00<?, ?it/s]


Error in checking for image issues: 'list' object has no attribute 'width'

Audit complete. 28741 issues found in the dataset.


In [13]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
near_duplicate       18365
       outlier       10376
       non_iid           0

Dataset Information: num_examples: 76126, num_classes: 1


------------------ near_duplicate issues -------------------

About this issue:
	A (near) duplicate issue refers to two or more examples in
    a dataset that are extremely similar to each other, relative
    to the rest of the dataset.  The examples flagged with this issue
    may be exactly duplicated, or lie atypically close together when
    represented as vectors (i.e. feature embeddings).
    

Number of examples with this issue: 18365
Overall dataset quality in terms of this issue: 0.0003

Examples representing most severe instances of this issue:
       is_near_duplicate_issue  near_duplicate_score                                                                                near_duplicate_sets  distance_to_nearest_neighbor
70849             